### Install necessary Packages

In [1]:
#!pip install "unstructured[all-docs]" unstructured-client watermark python-dotenv pydantic langchain langchain-community langchain_core langchain_openai chromadb

In [2]:
# Get installed versions
import watermark

%load_ext watermark
%watermark -n -v -m -g -b

Python implementation: CPython
Python version       : 3.10.15
IPython version      : 8.29.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 151 Stepping 2, GenuineIntel
CPU cores   : 20
Architecture: 64bit

Git hash: 

Git branch: 



### Import necessary packages

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Unstructured.io lookup

In [4]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common (package)
    csv
    doc
    docx
    email
    epub
    html (package)
    image
    json
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    c:\users\user\anaconda3\envs\fairlens\lib\site-packages\unstructured\partition\__init__.py




### Load PDF

In [5]:
from unstructured.partition.pdf import partition_pdf

pdf_path = "./data/Sample-Accounting-Income-Statement-PDF-File.pdf"

elements = partition_pdf(filename=pdf_path)

In [6]:
#elements

In [7]:
print(f"Length of elements: {len(elements)}")

Length of elements: 257


In [8]:
import json

element_dict = [element.to_dict() for element in elements]
output = json.dumps(element_dict, indent=2)

In [9]:
#print(output)

In [10]:
unique_element_type = set()

for element in element_dict:
    unique_element_type.add(element["type"])
print(unique_element_type)

{'UncategorizedText', 'Title', 'NarrativeText', 'Footer', 'Header', 'ListItem'}


### API CLient

In [11]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

unstructured_api_key = os.getenv("UNSTRUCTURED_API_KEY")
unstructured_api_url = os.getenv("UNSTRUCTURED_API_URL")

In [12]:
unstructured_api_key

'NexlrzZqBXLpvfS40owPzq5ttwucaE'

In [13]:
from unstructured_client import UnstructuredClient

client = UnstructuredClient(
                                api_key_auth=unstructured_api_key,
                                server_url=unstructured_api_url
                            )

In [14]:
client

### Extract Table With Client API

In [15]:
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.staging.base import dict_to_elements
from unstructured_client.models import shared, operations

In [16]:
with open(pdf_path, "rb") as f:
    files = shared.Files(
        content=f.read(),
        file_name=pdf_path
    )
    

req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=files,
        strategy="hi_res",
        hi_res_model_name="yolox",
        skip_infer_table_types=[],
        pdf_infer_table_structure=True
    )
)

try:
    resp = client.general.partition(request=req)
    elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)


INFO: HTTP Request: GET https://api.unstructuredapp.io/general/docs "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 200 OK"
